In [ ]:
import socket as so
import time
import struct
import csv

In [ ]:
def capture_packets(duration = 5):
  raw_socket = so.socket(so.AF_PACKET, so.SOCK_RAW, so.ntohs(3))

  start_time = time.time()
  packets = []

  while (time.time() - start_time < duration):
      raw_data, _ = raw_socket.recvfrom(65535)
      packet = extract_features(raw_data)
      packets.append(packet)

  return packets

In [ ]:
def extract_features(data):
  eth_header = struct.unpack("!6s6sH", data[:14])
  dest_mac = ":".join(f"{b:02x}" for b in eth_header[0])
  src_mac = ":".join(f"{b:02x}" for b in eth_header[1])

  protocol = eth_header[2]

  ip_header = data[14:34]
  src_ip = ".".join(map(str, ip_header[12:16]))
  dest_ip = ".".join(map(str, ip_header[16:20]))

  return{
      "src_ip": src_ip,
      "dest_ip": dest_ip,
      "src_mac": src_mac,
      "dest_mac": dest_mac,
      "packet_size": len(data),
      "protocol": protocol
  }

In [ ]:
def save_to_csv(packets, file_name = "network_logs.csv"):
  with open(file_name, "w", newline = "") as file:
    writer = csv.writer(file)
    writer.writerow(packets[0].keys())
    for packet in packets:
      writer.writerow(packet.values())

In [ ]:
packets = capture_packets(duration=5)
print(f"Captured {len(packets)} packets")
save_to_csv(packets)

Captured 349 packets
